In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

Using Theano backend.
WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
F:\anacondathree\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

168 documents
58 classes ['2-Tier', '4-tier', 'Access Manager', 'Active Workspace Client', 'CAD softwares', 'ERP interface', 'FCC', 'FMS', 'FSC', 'Jira', 'Jira packages', 'PLM', 'PLM software list', 'PLM systems', 'PTC Windchill', 'PTC Windchill uses', 'Preferences', 'SOA', 'Smart Chatbot', 'Smart Dashboard', 'SmartPort', 'Structure Manager', 'Teamcenter', 'Teamcenter X', 'Teamcenter customer', 'Teamcenter help', 'Teamcenter interfaces', 'Teamcenter rapid start', 'Teamcenter uses', 'Windchill PLM system', 'action handlers', 'benefits of PLM', 'change management', 'classification', 'cost-effective', 'data model', 'dataset', 'difference', 'document management', 'enterprise', 'goodbye', 'greeting', 'handlers', 'implement smartport', 'item', 'plm uses', 'pool manager', 'pricing model', 'revision rule', 'rich client', 'role of smartport', 'rule handlers', 'smartport features', 'teamcenter services', 'thanks', 'thin client', 'ticketing system', 'volume']
126 unique lemmatized words ["'s", '2

In [5]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        # output is a '0' for each tag and '1' for current tag (for each pattern)
        output_row = list(output_empty)
        output_row[classes.index(doc[1])] = 1
        training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-5-cd2d2c45d8a7>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [6]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=8, batch_size=100, verbose=1)
print("model created")

Epoch 1/8
21168/21168 [==============================] - 1915s 90ms/step - loss: 3.8807 - accuracy: 0.0934
Epoch 2/8
21168/21168 [==============================] - 2113s 100ms/step - loss: 3.0059 - accuracy: 0.3013
Epoch 3/8
21168/21168 [==============================] - 1921s 91ms/step - loss: 1.8987 - accuracy: 0.5223
Epoch 4/8
21168/21168 [==============================] - 2160s 102ms/step - loss: 1.2417 - accuracy: 0.6740
Epoch 5/8
21168/21168 [==============================] - 1985s 94ms/step - loss: 0.8716 - accuracy: 0.7688
Epoch 6/8
21168/21168 [==============================] - 1966s 93ms/step - loss: 0.6554 - accuracy: 0.8267
Epoch 7/8
21168/21168 [==============================] - 1818s 86ms/step - loss: 0.5292 - accuracy: 0.8604
Epoch 8/8
21168/21168 [==============================] - 1932s 91ms/step - loss: 0.4356 - accuracy: 0.8843
model created


In [7]:
model.save('chatbot_model.h5', hist)

In [8]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))